# Домашнее задание №1

В этом домашнем задании вы познакомитесь с платформой Kaggle, а также поучаствуете в небольшом сореврновании, которое поможет закрепить вам знания, полученные на наших лекциях.

Соревнование можно найти по ссылке: https://www.kaggle.com/c/mytracker-mlhomework1-fall20 \
Для того, чтобы присоединиться к этому соревнованию, перейдите по ссылке: https://www.kaggle.com/t/ebaddc30cce54e329c64f7504b274c2c \
Скачать данные можно во вкладке `Data`, в которой находится 3 файла:
 - `train.csv` - обучающая выборка
 - `test.csv` - тестовая выборка
 - `sampleSubmission.csv` - пример сабмита, который вы должны будете загрузить на Kaggle, чтобы предоставить свое решение (аналогичный файл мы с вами сгенерируем в этом ноутбуке)
 
Описание данных вы можете прочитать также на Kaggle.

Несколько основных моментов данного домашнего задания:
 1. Метрикой является RMSE (Root Mean Square Error, https://en.wikipedia.org/wiki/Root-mean-square_deviation).
 2. Тестовая выборка из файла `test.csv` разделена на 2 части: 
  - __Public Leaderboard__ - во время участия вы можете ориентироваться на этот лидерборд, чтобы оценивать ваше решение, а также видеть несколько наших бенчмарков и решения других участников.\
  **Важный момент**. Оценка на Public Leaderboard строится не на всех примерах из `test.csv`, а только на 60% (каких именно узнать нельзя).
  - __Private Leaderboard__ - откроется только после окончания соревнования. Именно по нему мы будем распределять баллы и он состоит из оставшихся 40% примеров из `test.csv`.
 3. Мы надеемся на то, что вы будете не только использовать модели машинного обучения, которые мы будем проходить по ходу лекций, но и попробуете сами изучить данные и сделать прогнозы "руками". Соревнование будет длиться достаточно долго (до 23.11), тем самым будет неплохо, если вы сможете изучить выданные датасеты, чтобы сделать более качественный прогноз.
 4. В день можно отправить до 5 посылок с решением, так что не оставляйте все на последний день.
 5. В конце соревнования ТОП10 участников мы попросим прислать рабочий ноутбук с их решением, чтобы убедиться, что никто не пытался сжульничать, так что держите свое решение "в чистоте", чтобы по окончанию соревнования его можно было показать нам.
 6. В соревновании нет ограничений на модели или подходы, которые вы будете использовать, если они не выходят за рамки данных, предоставленных в данном соревновании.

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import pandas_profiling
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import date
from fbprophet import Prophet

In [4]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
submission_filename = 'data/sampleSubmission.csv'

In [5]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [6]:
train['Date'] =pd.to_datetime(train.Date)
train.loc[:,'Holiday'].fillna(0, inplace=True)
train.head()


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,Holiday
0,474,5,2019-04-13,4256,502,1,0,0.0
1,862,5,2019-04-13,2574,319,1,0,0.0
2,108,5,2019-04-13,4687,492,1,0,0.0
3,48,5,2019-04-13,9779,1370,1,0,0.0
4,539,5,2019-04-13,2156,234,1,0,0.0


In [17]:
for_cycle=np.sort(train['Store'].unique())
print(for_cycle)

[   1    2    3 ... 1113 1114 1115]


In [40]:
predictions = 48

In [46]:
for i in for_cycle:
    train_df=train[(train['Store']== i)].loc[:,['Date','Sales']].reset_index()
    train_df=train_df.loc[:,['Date','Sales']]
    train_df.columns = ['ds', 'y']
    #print(train_df.head())
    m = Prophet(daily_seasonality=False, weekly_seasonality=True, yearly_seasonality=False)
    m.fit(train_df)
    future = m.make_future_dataframe(periods=predictions)
    forecast = m.predict(future)
    forecast_test = forecast.loc[:,['ds','trend']].tail(predictions)
    forecast_test['Store'] = i
    #print(forecast_test)
    if i == 1:
        union_f=forecast_test
    else:
        union_f = pd.concat([union_f, forecast_test])
    #if i == 3:
        #break
#print(union_f)

In [51]:
union_f = union_f.reset_index().drop('index', axis=1)

In [54]:
union_f.to_csv('prophet.csv', index=None)

In [55]:
union_f.columns = ['Date', 'Sales', 'Store']

In [62]:
test['Date'] =pd.to_datetime(test.Date)

In [64]:
s1 = pd.merge(test, union_f , how='left', on=['Store', 'Date'])
s1

,ID,Store,DayOfWeek,Date,Holiday,Sales
0,1,12,6,2019-04-14,0,6908.587134
1,2,12,0,2019-04-15,0,6907.427915
2,3,12,1,2019-04-16,0,6906.268696
3,4,12,2,2019-04-17,0,6905.109476
4,5,12,3,2019-04-18,0,6903.950257
...,...,...,...,...,...,...
13099,13100,1113,0,2019-05-27,0,4274.559843
13100,13101,1113,1,2019-05-28,0,4275.128287
13101,13102,1113,2,2019-05-29,0,4275.696731
13102,13103,1113,3,2019-05-30,0,4276.265175


In [66]:
s2 = pd.merge(s1, sum_train , how='left', on=['Store', 'DayOfWeek'])
s2

,ID,Store,DayOfWeek,Date,Holiday,Sales_x,Sales_y
0,1,12,6,2019-04-14,0,6908.587134,0.0
1,2,12,0,2019-04-15,0,6907.427915,10243.2
2,3,12,1,2019-04-16,0,6906.268696,9335.6
3,4,12,2,2019-04-17,0,6905.109476,8543.2
4,5,12,3,2019-04-18,0,6903.950257,7827.4
...,...,...,...,...,...,...,...
13099,13100,1113,0,2019-05-27,0,4274.559843,6681.6
13100,13101,1113,1,2019-05-28,0,4275.128287,4959.6
13101,13102,1113,2,2019-05-29,0,4275.696731,4659.8
13102,13103,1113,3,2019-05-30,0,4276.265175,4746.2


In [69]:
s2['Sales'] = np.where((s2['Sales_y'] == 0)
                     , 0, s2['Sales_x'])
s2

,ID,Store,DayOfWeek,Date,Holiday,Sales_x,Sales_y,Sales
0,1,12,6,2019-04-14,0,6908.587134,0.0,0.000000
1,2,12,0,2019-04-15,0,6907.427915,10243.2,6907.427915
2,3,12,1,2019-04-16,0,6906.268696,9335.6,6906.268696
3,4,12,2,2019-04-17,0,6905.109476,8543.2,6905.109476
4,5,12,3,2019-04-18,0,6903.950257,7827.4,6903.950257
...,...,...,...,...,...,...,...,...
13099,13100,1113,0,2019-05-27,0,4274.559843,6681.6,4274.559843
13100,13101,1113,1,2019-05-28,0,4275.128287,4959.6,4275.128287
13101,13102,1113,2,2019-05-29,0,4275.696731,4659.8,4275.696731
13102,13103,1113,3,2019-05-30,0,4276.265175,4746.2,4276.265175


In [81]:
train['Date'].agg([np.max, np.min])

amax   2019-04-13
amin   2016-11-01
Name: Date, dtype: datetime64[ns]

In [106]:
train[(train['Date']> '2018-04-13') & (train['Date']< '2018-05-31')]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,Holiday
326850,474,2,2018-05-30,5487,587,1,1,0.0
326851,862,2,2018-05-30,6308,768,1,1,0.0
326852,108,2,2018-05-30,8399,809,1,1,0.0
326853,48,2,2018-05-30,10032,1288,1,1,0.0
326854,539,2,2018-05-30,5310,649,1,1,0.0
...,...,...,...,...,...,...,...,...
373850,73,5,2018-04-14,3230,260,1,0,0.0
373851,649,5,2018-04-14,8054,722,1,0,0.0
373852,429,5,2018-04-14,6209,637,1,0,0.0
373853,898,5,2018-04-14,23130,3742,1,0,0.0


In [109]:
train[(train['Date']> '2018-04-13') & (train['Date']< '2018-05-31')].loc[:,['Open','Date']].groupby('Date').agg([np.max,np.min,np.sum])

Open           
           amax amin   sum
Date                      
2018-04-14    1    0  1113
2018-04-15    1    0    32
2018-04-16    1    0  1113
2018-04-17    1    0  1113
2018-04-18    1    0  1113
...         ...  ...   ...
2018-12-25    1    0  1113
2018-12-26    1    0  1113
2018-12-27    1    0  1113
2018-12-28    1    0  1113
2018-12-29    1    0  1113

[260 rows x 3 columns]

In [47]:
sum_train = (train[['Store','DayOfWeek','Sales']]
                 .groupby(['Store','DayOfWeek'])
                 .agg(percentile(60))
                 .reset_index()
            )
sum_train

,Store,DayOfWeek,Sales
0,1,0,6140.0
1,1,1,5132.0
2,1,2,4963.0
3,1,3,4636.0
4,1,4,6059.4
...,...,...,...
7800,1115,2,6377.6
7801,1115,3,6043.2
7802,1115,4,6691.6
7803,1115,5,6682.8


In [123]:
sum_train_period = (train[(train['Date']> '2018-04-13') & (train['Date']< '2018-05-31')][['Store','DayOfWeek','Sales']]
                 .groupby(['Store','DayOfWeek'])
                 .agg(percentile(60))
                 .reset_index()
            )
sum_train_period.head()

,Store,DayOfWeek,Sales
0,1,0,7243.0
1,1,1,5081.0
2,1,2,5215.2
3,1,3,2040.6
4,1,4,6574.4


In [71]:
test['Date'] =pd.to_datetime(test.Date)

In [80]:
test['Date'].agg([np.max, np.min])

amax   2019-05-31
amin   2019-04-14
Name: Date, dtype: datetime64[ns]

In [30]:
test.head()

,ID,Store,DayOfWeek,Date,Holiday
0,1,12,6,2019-04-14,0
1,2,12,0,2019-04-15,0
2,3,12,1,2019-04-16,0
3,4,12,2,2019-04-17,0
4,5,12,3,2019-04-18,0


In [73]:
test.describe()

,ID,Store,DayOfWeek,Holiday
count,13104.000000,13104.000000,13104.000000,13104.0
mean,6552.500000,588.311355,2.958333,0.0
std,3782.943299,318.998619,1.999642,0.0
min,1.000000,12.000000,0.000000,0.0
25%,3276.750000,294.000000,1.000000,0.0
50%,6552.500000,600.000000,3.000000,0.0
75%,9828.250000,862.000000,5.000000,0.0
max,13104.000000,1113.000000,6.000000,0.0


In [124]:
s1 = pd.merge(test,sum_train_period , how='left', on=['Store', 'DayOfWeek'])
s1

,ID,Store,DayOfWeek,Date,Holiday,Sales
0,1,12,6,2019-04-14,0,0.0
1,2,12,0,2019-04-15,0,10454.8
2,3,12,1,2019-04-16,0,9166.0
3,4,12,2,2019-04-17,0,8888.4
4,5,12,3,2019-04-18,0,7273.0
...,...,...,...,...,...,...
13099,13100,1113,0,2019-05-27,0,8350.6
13100,13101,1113,1,2019-05-28,0,5638.8
13101,13102,1113,2,2019-05-29,0,5727.6
13102,13103,1113,3,2019-05-30,0,5480.0


In [74]:
submission = test['ID'].to_frame()
submission['prediction'] = s2['Sales']
submission.head()

,ID,prediction
0,1,0.000000
1,2,6907.427915
2,3,6906.268696
3,4,6905.109476
4,5,6903.950257


In [73]:
submission.to_csv('sampleSubmission.csv', index=None)